In [15]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import re


def fetch_and_filter_housing(service_key, output_path):
    """
    API 직접 호출 → DataFrame → 주거 정책 필터링 → CSV 저장
    """

    BASE_URL = "https://apis.data.go.kr/B554287/LocalGovernmentWelfareInformations/LcgvWelfarelist"

    params = {
        "serviceKey": service_key,   # Encoding 키
        "pageNo": 1,
        "numOfRows": 100,
        "lifeArray": "004",          # 청년
        "ctpvNm": "서울특별시"
    }

    # =========================
    # 1. API 호출
    # =========================
    response = requests.get(BASE_URL, params=params, timeout=30)

    print("HTTP Status:", response.status_code)

    if response.status_code != 200:
        print("❌ API 호출 실패")
        print(response.text)
        return

    xml_text = response.text

    # =========================
    # 2️. XML 파싱
    # =========================
    root = ET.fromstring(xml_text)

    total_count = root.findtext(".//totalCount")
    result_code = root.findtext(".//resultCode")
    result_msg = root.findtext(".//resultMessage")

    print("totalCount:", total_count)
    print("resultCode:", result_code)
    print("resultMessage:", result_msg)

    if result_code != "0":
        print("❌ API 응답 오류")
        return

    records = root.findall(".//servList")

    rows = []
    for rec in records:
        row = {}
        for child in list(rec):
            row[child.tag] = (child.text or "").strip()
        rows.append(row)

    df = pd.DataFrame(rows)
    print(f"\nXML 파싱 완료: {len(df)}건")

    # =========================
    # 3️. 주거 키워드 필터링
    # =========================
    housing_keywords = [
        "주거", "주택", "임대", "공공임대", "전세", "월세", "전월세", "임차",
        "보증금", "임차보증금", "주거비", "이사비", "중개수수료",
        "대출이자", "전세자금", "청년주택", "보증", "반환보증"
    ]

    pattern = re.compile("|".join(map(re.escape, housing_keywords)))

    df["__text__"] = (
        df.get("servNm", "").fillna("") + " " +
        df.get("servDgst", "").fillna("")
    )

    mask = df["__text__"].str.contains(pattern, na=False)
    df_housing = df.loc[mask].copy()

    print(f"\n주거로 판정된 정책: {len(df_housing)}건")

    # =========================
    # 4️. 필요한 컬럼만
    # =========================
    out_cols = [
        "servNm",
        "servDgst",
        "intrsThemaNmArray",
        "ctpvNm",
        "sggNm",
        "servId",
        "servDtlLink"
    ]

    out_cols = [c for c in out_cols if c in df_housing.columns]
    df_housing = df_housing[out_cols]

    df_housing.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"\n 저장 완료: {output_path}")
    print(df_housing.head(5))


# =====================================================
# ▶ 실행
# =====================================================
if __name__ == "__main__":

    SERVICE_KEY = "U4EKdYXkz7iVyJilo/7FVsxukjzFgSQ3/MOsmDEKWE6FPEYJ0XR+9nUgxbdqA0uqXJyDwZ/OyhBAbl61b0CZ6A=="

    fetch_and_filter_housing(
        service_key=SERVICE_KEY,
        output_path="filtered_api_data.csv"
    )


HTTP Status: 200
totalCount: 95
resultCode: 0
resultMessage: SUCCESS

XML 파싱 완료: 95건

주거로 판정된 정책: 7건

 저장 완료: filtered_api_data.csv
                       servNm  \
1           자녀출산 무주택가구 주거비 지원   
23    서울시 저소득 중증장애인 전세주택 제공사업   
36  강남구 신혼부부 청년 전월세 대출이자 지원사업   
43                  희망두배 청년통장   
58            신혼부부 임차보증금 지원사업   

                                             servDgst   intrsThemaNmArray  \
1   무주택 가구의 주거비 부담을 줄여 안정된 주거환경 속에서 자녀를 출산 할 수 있도록...  서민금융, 입양·위탁, 임신·출산   
23  저소득 중증장애인이 월세 부담에서 벗어나 전세주택에 입주할 수 있도록  최장 6년간...                 NaN   
36                         강남구 거주 신혼부부 및 청년 주거비 부담 완화                서민금융   
43  학자금 대출, 주거비, 비정규직 취업 등으로 어려움을 겪고 있는 청년 대상 자산형성...                서민금융   
58  신혼부부의 주거비 부담 경감 및 주거안정성 강화를 위해 임차보증금 대출을 지원하여 ...                서민금융   

   ctpvNm sggNm       servId  \
1   서울특별시   NaN  WLF00006181   
23  서울특별시   은평구  WLF00002109   
36  서울특별시   강남구  WLF00005414   
43  서울특별시   NaN  WLF00005247   
58  서울특별시   NaN  WLF00002617   

             